<a href="https://colab.research.google.com/github/janbanot/msc-cs-code/blob/main/sem3/DL/DL_2025_Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

UGZSN - Sprawozdanie 2 - Jan Banot

In [ ]:
!uv pip install torchinfo

In [ ]:
import math
import inspect
import random
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F
from dataclasses import dataclass
from torch.nn.utils.rnn import pad_sequence
import matplotlib.pyplot as plt

# --- ARCHITEKTURA GPT ---
class CausalSelfAttention(nn.Module):
    """
    Przyczynowa samouwaga (Causal Self-Attention).
    Token może "patrzeć" tylko na tokeny z przeszłości, tj. za nim, a nie po nim.
    """

    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0

        # Jedna warstwa liniowa liczy naraz Q, K i V (potem rozdzielamy na 3 części).
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd, bias=config.bias)

        # Projekcja wyjściowa po scaleniu głowic
        self.c_proj = nn.Linear(config.n_embd, config.n_embd, bias=config.bias)

        # Dropout na wyjściu bloku uwagi
        self.resid_dropout = nn.Dropout(config.dropout)
        self.attn_dropout_p = float(config.dropout)

        self.n_head = config.n_head
        self.n_embd = config.n_embd

    def forward(self, x):
        # x: (B, T, C) -> batch, długość sekwencji, wymiar embeddingu
        B, T, C = x.size()
        head_size = C // self.n_head

        # Liczymy Q, K, V i rozdzielamy wynik na trzy tensory
        q, k, v = self.c_attn(x).split(self.n_embd, dim=2)

        # (B, T, C) -> (B, n_head, T, head_size)
        # .transpose(1, 2) przenosi wymiar głowic przed czas
        q = q.view(B, T, self.n_head, head_size).transpose(1, 2)
        k = k.view(B, T, self.n_head, head_size).transpose(1, 2)
        v = v.view(B, T, self.n_head, head_size).transpose(1, 2)

        # PyTorch 2.0+: scaled_dot_product_attention może użyć Flash Attention.
        # is_causal=True wymusza maskę trójkątną (brak wglądu w przyszłość)
        y = F.scaled_dot_product_attention(
            q, k, v,
            attn_mask=None,
            dropout_p=self.attn_dropout_p if self.training else 0.0,
            is_causal=True,
        )

        # Scal głowice: (B, n_head, T, head_size) -> (B, T, C)
        y = y.transpose(1, 2).contiguous().view(B, T, C)

        # Projekcja + dropout rezydualny
        return self.resid_dropout(self.c_proj(y))


class MLP(nn.Module):
    """
    Prosty MLP działający niezależnie na każdym tokenie
    """
    def __init__(self, config):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(config.n_embd, 4 * config.n_embd, bias=config.bias),
            nn.GELU(),
            nn.Linear(4 * config.n_embd, config.n_embd, bias=config.bias),
            nn.Dropout(config.dropout),
        )

    def forward(self, x):
        return self.net(x)


class Block(nn.Module):
    """
    Pojedynczy blok Transformera (pre-norm):
    LN -> Attention -> resid, LN -> MLP -> resid
    """

    def __init__(self, config):
        super().__init__()
        self.ln_1 = nn.LayerNorm(config.n_embd, elementwise_affine=True, bias=config.bias)
        self.attn = CausalSelfAttention(config)
        self.ln_2 = nn.LayerNorm(config.n_embd, elementwise_affine=True, bias=config.bias)
        self.mlp = MLP(config)

    def forward(self, x):
        # Pre-norm: najpierw normalizacja, potem operacja, potem dodanie rezydualne
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x

@dataclass
class GPTConfig:
    block_size: int = 128  # Zapas dla 15+15+16 bitów + znaki specjalne
    vocab_size: int = 7    # 0, 1, +, =, PAD, EOS, ewentualnie separator
    n_layer: int = 4
    n_head: int = 8
    n_embd: int = 128      # To da nam ok. 800k - 1M parametrów
    dropout: float = 0.1
    bias: bool = True

class GPT(nn.Module):
    def __init__(self, config: GPTConfig):
        super().__init__()
        assert config.vocab_size is not None
        assert config.block_size is not None
        self.config = config

        self.transformer = nn.ModuleDict(dict(
            wte=nn.Embedding(config.vocab_size, config.n_embd),    # embedding tokenów
            wpe=nn.Embedding(config.block_size, config.n_embd),    # embedding pozycji
            drop=nn.Dropout(config.dropout),
            h=nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f=nn.LayerNorm(config.n_embd, bias=config.bias),
        ))

        # Głowa językowa: projekcja na rozmiar słownika
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

        # Weight tying: wte.weight i lm_head.weight to ten sam parametr
        # Zmniejsza liczbę parametrów i zwykle poprawia jakość.
        self.transformer.wte.weight = self.lm_head.weight

        # Bufor z indeksami pozycji: unikamy torch.arange w każdym forward()
        # persistent=False => nie zapisuje się do checkpointów (bo można odtworzyć)
        self.register_buffer(
            "pos_idx",
            torch.arange(config.block_size, dtype=torch.long),
            persistent=False,
        )

        # Inicjalizacja wag
        self.apply(self._init_weights)

        # Specjalna inicjalizacja dla projekcji rezydualnych (jak w GPT-2),
        # aby stabilizować wariancję w głębokiej sieci na starcie treningu
        for pn, p in self.named_parameters():
            if pn.endswith("c_proj.weight"):
                torch.nn.init.normal_(p, mean=0.0, std=0.02 / math.sqrt(2 * config.n_layer))

        print("Liczba parametrów: %.2fM" % (self.get_num_params() / 1e6,))

    def get_num_params(self) -> int:
        """Zwraca liczbę parametrów (bez embeddingów pozycyjnych)."""
        n_params = sum(p.numel() for p in self.parameters())
        n_params -= self.transformer.wpe.weight.numel()
        return n_params

    def _init_weights(self, module: nn.Module) -> None:
        """Domyślna inicjalizacja wag (rozkład normalny)."""
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.01)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.01)

    def forward(self, idx: torch.Tensor, targets: torch.Tensor | None = None):
        device = idx.device
        b, t = idx.size()
        assert t <= self.config.block_size, (
            f"Sekwencja {t} jest dłuższa niż block_size={self.config.block_size}"
        )

        # (b, t, n_embd) + (t, n_embd) => broadcast po batchu
        tok_emb = self.transformer.wte(idx)

        pos = self.pos_idx[:t].to(device)   # Pozycje kolejnych tokenów
        pos_emb = self.transformer.wpe(pos) # Model uczy się kodowania pozycji

        x = self.transformer.drop(tok_emb + pos_emb)  # Opcjonalny dropout

        for block in self.transformer.h:
            x = block(x)
        x = self.transformer.ln_f(x)

        if targets is not None:
            # Trening: logity dla wszystkich pozycji + loss.
            logits = self.lm_head(x)  # (b, t, vocab)

            # cross_entropy oczekuje (N, C, ...) dla logits oraz (N, ...) dla targetów,
            # więc przestawiamy osie na (b, vocab, t)
            loss = F.cross_entropy(
                logits.transpose(1, 2),
                targets,
                ignore_index=-1,  # Ignoruj tokeny o tej wartości
            )
        else:
            # Inferencja: interesuje nas predykcja następnego tokena (ostatnia pozycja)
            logits = self.lm_head(x[:, [-1], :])  # (b, 1, vocab)
            loss = None

        return logits, loss

    def configure_optimizers(self, weight_decay, learning_rate, betas, device_type):
        """
        AdamW z podziałem parametrów na:
        - decay: wagi warstw liniowych (nn.Linear.weight)
        - no_decay: biasy, LayerNorm, Embeddingi
        """

        whitelist = (nn.Linear,)
        blacklist = (nn.LayerNorm, nn.Embedding)

        # mapowanie: id(param) -> (param, czy_decay)
        # czy_decay = True oznacza, że parametr trafi do grupy z weight_decay
        param_to_decay = {}
        id_to_param = {}

        for module in self.modules():
            for name, param in module.named_parameters(recurse=False):
                if not param.requires_grad:
                    continue

                pid = id(param)
                id_to_param[pid] = param

                # Reguły klasyfikacji:
                is_bias = name.endswith("bias")
                is_linear_weight = isinstance(module, whitelist) and name.endswith("weight")
                is_blacklisted = isinstance(module, blacklist)

                # Priorytet: NO_DECAY wygrywa zawsze (szczególnie ważne przy weight tying)
                if is_bias or is_blacklisted:
                    param_to_decay[pid] = False
                elif is_linear_weight:
                    # decay tylko jeśli parametr nie został wcześniej oznaczony jako no_decay
                    param_to_decay.setdefault(pid, True)
                else:
                    # bezpieczny domyślny wybór: brak decay
                    param_to_decay.setdefault(pid, False)

        decay_params = [id_to_param[pid] for pid, dec in param_to_decay.items() if dec]
        nodecay_params = [id_to_param[pid] for pid, dec in param_to_decay.items() if not dec]

        optim_groups = [
            {"params": decay_params, "weight_decay": weight_decay},
            {"params": nodecay_params, "weight_decay": 0.0},
        ]

        # device_type: jeśli przekazujesz torch.device, użyj device_type = device.type
        if isinstance(device_type, torch.device):
            device_type = device_type.type

        use_fused = (device_type == "cuda") and ("fused" in inspect.signature(torch.optim.AdamW).parameters)
        print(f"Używanie fused AdamW: {use_fused}")

        return torch.optim.AdamW(
            optim_groups,
            lr=learning_rate,
            betas=betas,
            fused=use_fused,
        )

    @torch.no_grad()
    def generate(self, idx: torch.Tensor, max_new_tokens: int, temperature: float = 1.0):
        """
        Generuje kolejne tokeny autoregresyjnie

        Ważne:
        - Zapamiętujemy poprzedni tryb (train/eval) i przywracamy go na końcu,
          żeby generate() nie psuło treningu, jeśli zostanie wywołane w trakcie.
        - @torch.no_grad() wyłącza gradienty (szybciej i mniej pamięci)
        """
        was_training = self.training
        self.eval()
        try:
            for _ in range(max_new_tokens):
                # Ograniczamy kontekst do block_size (model nie widzi dalej).
                idx_cond = idx[:, -self.config.block_size:]

                # forward() w trybie inferencji zwraca logity tylko dla ostatniej pozycji (b, 1, vocab)
                logits, _ = self(idx_cond)
                logits = logits[:, -1, :]  # (b, vocab)

                # temperature=0 -> deterministycznie (argmax)
                if temperature == 0.0:
                    idx_next = torch.argmax(logits, dim=-1, keepdim=True)
                else:
                    logits = logits / temperature
                    probs = F.softmax(logits, dim=-1)
                    idx_next = torch.multinomial(probs, num_samples=1)

                idx = torch.cat([idx, idx_next], dim=1)

            return idx
        finally:
            # Przywróć poprzedni tryb modelu
            self.train(was_training)

In [ ]:
VOCAB = {'0': 0, '1': 1, '+': 2, '=': 3, '<PAD>': 4, '<EOS>': 5}
INV_VOCAB = {v: k for k, v in VOCAB.items()}
PAD_IDX = VOCAB['<PAD>']
EOS_IDX = VOCAB['<EOS>']

def encode(tokens):
    return [VOCAB[t] for t in tokens]

def decode(tokens):
    return ''.join(INV_VOCAB[t] for t in tokens if t in INV_VOCAB)

def make_addition_example(variant, max_bits=15):
    # 1. Losujemy dwie liczby
    a_int = random.randint(0, 2**max_bits - 1)
    b_int = random.randint(0, 2**max_bits - 1)
    c_int = a_int + b_int

    a_bin = bin(a_int)[2:]
    b_bin = bin(b_int)[2:]
    c_bin = bin(c_int)[2:]

    if variant == 1:
        # Stała liczba bitów (dopełnienie zerami do 15/16)
        a_str = a_bin.zfill(max_bits)
        b_str = b_bin.zfill(max_bits)
        c_str = c_bin.zfill(max_bits + 1)
    elif variant == 2:
        # Minimalna liczba bitów
        a_str, b_str, c_str = a_bin, b_bin, c_bin
    elif variant == 3:
        # Odwrócona kolejność bitów (Najmniej znaczący bit pierwszy)
        a_str, b_str, c_str = a_bin[::-1], b_bin[::-1], c_bin[::-1]

    # Konstrukcja ciągu: A + B = C <EOS>
    input_str = a_str + '+' + b_str + '='
    full_tokens = list(input_str) + list(c_str) + ['<EOS>']

    return torch.tensor(encode(full_tokens), dtype=torch.long), input_str, c_str

In [ ]:
def evaluate_model(model, variant, n_examples=1000, max_bits=15):
    model.eval()
    device = next(model.parameters()).device
    correct = 0

    for _ in range(n_examples):
        ids, prefix, target_suffix = make_addition_example(variant, max_bits)
        prompt_ids = encode(list(prefix))
        prompt_tensor = torch.tensor(prompt_ids, dtype=torch.long).unsqueeze(0).to(device)

        # Generujemy tyle tokenów, ile ma wynik + EOS
        max_gen = len(target_suffix) + 1
        out = model.generate(prompt_tensor, max_new_tokens=max_gen, temperature=0.0)

        gen_str = decode(out[0, len(prompt_ids):].tolist())
        if target_suffix + '<EOS>' in gen_str or target_suffix == gen_str.replace('<EOS>',''):
            correct += 1

    return correct / n_examples

def train_variant(variant_idx, total_steps=15000):
    print(f"\n--- Trenowanie Wariantu {variant_idx} ---")
    config = GPTConfig()
    model = GPT(config).to(device)

    optimizer = model.configure_optimizers(weight_decay=0.01, learning_rate=5e-4, betas=(0.9, 0.95), device_type=device)
    loss_history = []

    for step in range(total_steps):
        # Generowanie batcha
        batch = [make_addition_example(variant_idx) for _ in range(256)]
        xs_list = [item[0][:-1] for item in batch]
        ys_list = [item[0][1:] for item in batch]

        xs = pad_sequence(xs_list, batch_first=True, padding_value=PAD_IDX).to(device)
        ys = pad_sequence(ys_list, batch_first=True, padding_value=-1).to(device)

        logits, loss = model(xs, ys)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        loss_history.append(loss.item())
        if step % 500 == 0:
            print(f"Krok {step}, Loss: {loss.item():.4f}")

    acc = evaluate_model(model, variant_idx)
    print(f"Finalna dokładność (Wariant {variant_idx}): {acc*100:.2f}%")
    return loss_history, acc

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
results = {}

for v in [1, 2, 3]:
    loss_h, final_acc = train_variant(v)
    results[v] = {'loss': loss_h, 'acc': final_acc}

# Wykresy
plt.figure(figsize=(12, 6))
for v in [1, 2, 3]:
    plt.plot(results[v]['loss'], label=f'Wariant {v} (Acc: {results[v]["acc"]*100:.1f}%)')
plt.xlabel('Krok')
plt.ylabel('Loss')
plt.legend()
plt.title('Porównanie procesów uczenia dla 3 wariantów dodawania')
plt.show()

Analiza wyników i odpowiedzi na pytania
1. Który z wariantów problemu jest najłatwiejszy do rozwiązania i dlaczego?
Najłatwiejszym wariantem okazał się Wariant 3 (minimalna liczba bitów, kolejność odwrócona), który osiągnął 100.0% dokładności.

Dlaczego? Wynika to z mechaniki dodawania pisemnego. Dodając dwie liczby, zaczynamy od najmniej znaczącego bitu (LSB), ponieważ musimy wiedzieć, czy występuje "przeniesienie" (carry) do następnej kolumny. W wariancie 3 model dostaje bity i generuje wynik dokładnie w tej samej kolejności, w jakiej działają prawa arytmetyki. Dzięki temu sieć może łatwo "pamiętać" bit przeniesienia z poprzedniego kroku i wykorzystać go w bieżącym.

2. Czy wariant 3. ułatwia trening w stosunku do wariantu 2., jeżeli tak to dlaczego?
Tak, wariant 3 znacząco ułatwia trening (skok dokładności z 89.4% na 100.0% oraz szybszy spadek funkcji straty).

Uzasadnienie: W wariancie 2 (kolejność standardowa, od MSB) model musi przewidzieć pierwszy (najbardziej znaczący) bit wyniku, nie wiedząc jeszcze, co stanie się na samym końcu liczby. Jednak pierwszy bit wyniku może zależeć od przeniesienia, które powstało na samym końcu i "przeszło" przez całą długość liczby (tzw. carry propagation). Jest to problem zależności dalekosiężnej, który jest trudny dla sieci. W wariancie 3 problem ten znika. Przeniesienie jest zawsze dostępne "pod ręką", z poprzedniego wygenerowanego tokena.

3. Komentarz do otrzymanych wyników
Wariant 3 (100%): Model perfekcyjnie opanował algorytm dodawania. Stabilizacja straty nastąpiła bardzo szybko (już około 2500 kroku strata spadła poniżej 0.5 i pozostała stabilna).

Wariant 2 (89.4%): Model radzi sobie dobrze, ale popełnia błędy, prawdopodobnie przy bardzo długich przeniesieniach, które muszą być przewidziane "z góry".

Wariant 1 (32.7%): To najciekawszy wynik - stała liczba 15 bitów okazała się najtrudniejsza. Prawdopodobnie wynika to z faktu, że przy stałej długości (dopełnianie zerami) sekwencje są zawsze długie, a istotne dane są "rozmyte" w dużej ilości zer. Atencja modelu musi przeszukiwać znacznie dłuższy kontekst, co przy ograniczonej liczbie parametrów (0.79M) i kroków treningowych utrudniło znalezienie poprawnej reguły.